In [1]:
import numpy as np
import pandas as pd
import geopandas
import libpysal
import momepy
import scipy

from dask.distributed import Client, LocalCluster, as_completed

In [2]:
workers = 8
client = Client(LocalCluster(n_workers=workers, threads_per_worker=1))
client

Client Scheduler: tcp://127.0.0.1:43135 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 8 Memory: 84.28 GB


In [3]:
cross_chunk = pd.read_parquet('../../urbangrammar_samba/spatial_signatures/cross-chunk_indices.pq')

In [4]:
def generate_w(chunk_id):
    # load cells of a chunk
    cells = geopandas.read_parquet(f"../../urbangrammar_samba/spatial_signatures/morphometrics/cells/cells_{chunk_id}.pq")
    
    # add neighbouring cells from other chunks
    cross_chunk_cells = []
    
    for chunk, inds in cross_chunk.loc[chunk_id].indices.iteritems():
        add_cells = geopandas.read_parquet(f"../../urbangrammar_samba/spatial_signatures/morphometrics/cells/cells_{chunk}.pq").iloc[inds]
        cross_chunk_cells.append(add_cells)
    
    df = cells.append(pd.concat(cross_chunk_cells, ignore_index=True), ignore_index=True)

    w = libpysal.weights.Queen.from_dataframe(df, geom_col='tessellation')
    w3 = momepy.sw_high(k=3, weights=w)
    
    scipy.sparse.save_npz(f"../../urbangrammar_samba/spatial_signatures/weights/w_{chunk_id}.npz", w.sparse)
    scipy.sparse.save_npz(f"../../urbangrammar_samba/spatial_signatures/weights/w3_{chunk_id}.npz", w3.sparse)
    
    return f"Chunk {chunk_id} processed sucessfully."

In [5]:
%%time
inputs = iter(range(103))
futures = [client.submit(generate_w, next(inputs)) for i in range(workers)]
ac = as_completed(futures)
for finished_future in ac:
    # submit new future 
    try:
        new_future = client.submit(generate_w, next(inputs))
        ac.add(new_future)
    except StopIteration:
        pass
    print(finished_future.result())

Chunk 0 processed sucessfully.
Chunk 3 processed sucessfully.
Chunk 2 processed sucessfully.
Chunk 1 processed sucessfully.
Chunk 7 processed sucessfully.
Chunk 4 processed sucessfully.
Chunk 5 processed sucessfully.
Chunk 8 processed sucessfully.
Chunk 9 processed sucessfully.
Chunk 11 processed sucessfully.
Chunk 10 processed sucessfully.
Chunk 6 processed sucessfully.
Chunk 13 processed sucessfully.
Chunk 14 processed sucessfully.
Chunk 12 processed sucessfully.
Chunk 15 processed sucessfully.
Chunk 16 processed sucessfully.
Chunk 18 processed sucessfully.
Chunk 17 processed sucessfully.
Chunk 19 processed sucessfully.
Chunk 20 processed sucessfully.
Chunk 21 processed sucessfully.
Chunk 23 processed sucessfully.
Chunk 22 processed sucessfully.
Chunk 25 processed sucessfully.
Chunk 24 processed sucessfully.
Chunk 26 processed sucessfully.
Chunk 28 processed sucessfully.
Chunk 27 processed sucessfully.
Chunk 29 processed sucessfully.
Chunk 31 processed sucessfully.
Chunk 30 processed

In [6]:
client.close()